## Gemini 멀티모달리티 분석

### 라이브러리 설치

In [1]:
%pip install --upgrade --quiet google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.5/218.5 kB 7.3 MB/s eta 0:00:00


### GCP 환경설정 및 로그인

In [1]:
import os

PROJECT_ID = "ai-hangsik" #@param {type:"string"}
REGION = "us-central1" #@param {type:"string"}
USE_VERTEX_AI = True #@param {type:"boolean"}

In [ ]:
!gcloud auth application-default login
!gcloud auth application-default set-quota-project {PROJECT_ID}

### Vertex AI Client 실행

In [11]:
import base64
from IPython.display import Image, display, Markdown

from google import genai
from google.genai import types
from google.genai.types import HttpOptions

client = genai.Client(
    vertexai=USE_VERTEX_AI,
    project=PROJECT_ID,
    location=REGION,)

### Gemini 멀티모달 분석 함수

In [16]:
def read_bytes_file(file_path):
  with open(file_path, "rb") as f:
    return f.read()

def generate(prompt:str,
             file_name:str,
             mime_type:str,
             ):

  # 분석 대상 bytes 로 읽기.
  file_bytes = read_bytes_file(file_path=file_name)

  contents = [
    types.Content(
      role="user",
      parts=[
        types.Part.from_bytes( # from_bytes 함수로 처리
            data=base64.b64decode(base64.b64encode(file_bytes)),
            mime_type=mime_type),
        types.Part.from_text(  # Prompt.
            text=prompt)
      ]
    )
  ]

  response = client.models.generate_content(
      model="gemini-2.5-flash",
      contents= contents,
  )
  return response


### 동영상 분석

#### 동영상 다운로드

In [17]:
video_file_name = "ForBiggerBlazes.mp4"
url = "http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/ForBiggerBlazes.mp4"
!wget $url

--2025-07-24 02:11:29--  http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/ForBiggerBlazes.mp4
Resolving commondatastorage.googleapis.com (commondatastorage.googleapis.com)... 74.125.139.207, 173.194.210.207, 173.194.212.207, ...
Connecting to commondatastorage.googleapis.com (commondatastorage.googleapis.com)|74.125.139.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2498125 (2.4M) [video/mp4]
Saving to: ‘ForBiggerBlazes.mp4.1’

ForBiggerBlazes.mp4 100%[===================>]   2.38M  --.-KB/s    in 0.008s  

2025-07-24 02:11:29 (310 MB/s) - ‘ForBiggerBlazes.mp4.1’ saved [2498125/2498125]



#### Gemini 동영상 분석

In [18]:
prompt = "Video 내용을 한국어로 요약해주세요. "

response = generate(prompt=prompt,
                    file_name=video_file_name,
                    mime_type="video/mp4"
                    )

display(Markdown(response.text))

이 영상은 태블릿에서 시청하던 콘텐츠를 TV로 전송하는 모습을 보여줍니다.

한 사람이 '왕좌의 게임'을 태블릿으로 시청하다가 화면을 터치하자, 즉시 해당 영상이 대형 TV 화면으로 전송되어 더 크고 몰입감 있는 시청 경험을 제공합니다.

이어서 '크롬캐스트' 기기를 TV의 HDMI 포트에 연결하는 장면이 나오며, HBO GO 콘텐츠를 TV에서 시청할 수 있음을 강조합니다.

마지막으로, 크롬캐스트의 가격이 35달러이며 HBO 구독이 필요하다고 안내합니다.

### 이미지 분석

#### 이미지 다운로드

In [19]:
image_file_name = "ForBiggerBlazes.jpg"
url = "http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/images/ForBiggerBlazes.jpg"
!wget $url

--2025-07-24 02:12:05--  http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/images/ForBiggerBlazes.jpg
Resolving commondatastorage.googleapis.com (commondatastorage.googleapis.com)... 74.125.139.207, 173.194.210.207, 173.194.212.207, ...
Connecting to commondatastorage.googleapis.com (commondatastorage.googleapis.com)|74.125.139.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28481 (28K) [image/jpeg]
Saving to: ‘ForBiggerBlazes.jpg’

ForBiggerBlazes.jpg 100%[===================>]  27.81K  --.-KB/s    in 0s      

2025-07-24 02:12:05 (327 MB/s) - ‘ForBiggerBlazes.jpg’ saved [28481/28481]



#### Gemini 이미지 분석

In [20]:
prompt = "이미지를 설정해주고 이미지에 적힌 글씨를 알려주세요. "

response = generate(prompt=prompt,
                    file_name=image_file_name,
                    mime_type="image/jpeg"
                    )

display(Markdown(response.text))

이미지에 적힌 글씨는 다음과 같습니다:

**FOR BIGGER BLAZES**
**Game of Thrones on HBO GO**

### 동영상과 이미지를 동시처리

In [21]:
def generate2(prompt:str,
             image_file_name:str,
             video_file_name:str,
             image_mime_type:str,
             video_mime_type:str,
             ):

  # 분석 대상 bytes 로 읽기.
  image_file_bytes = read_bytes_file(file_path=image_file_name)
  video_file_bytes = read_bytes_file(file_path=video_file_name)

  contents = [
    types.Content(
      role="user",
      parts=[
        types.Part.from_bytes( # from_bytes 함수로 처리
            data=base64.b64decode(base64.b64encode(image_file_bytes)),
            mime_type=image_mime_type),

        types.Part.from_bytes( # from_bytes 함수로 처리
            data=base64.b64decode(base64.b64encode(video_file_bytes)),
            mime_type=video_mime_type),

        types.Part.from_text(  # Prompt.
            text=prompt)
      ]
    )
  ]

  response = client.models.generate_content(
      model="gemini-2.5-flash",
      contents= contents,
  )
  return response


In [22]:
prompt = "동영상내에서 첨부된 이미지가 나타나는 시간을 알려주세요. "

response = generate2(prompt=prompt,
                    image_file_name=image_file_name,
                    video_file_name=video_file_name,
                    image_mime_type="image/jpeg",
                    video_mime_type="video/mp4"
                    )

display(Markdown(response.text))

첨부된 이미지는 동영상에서 **0:05**에 나타납니다.

## End of Document